### CNN  With TensorFlow's Estimator Class
- [Estimator Class](https://www.tensorflow.org/guide/estimators) is high level API for TensorFlow that simplifies training, eval, prediction, and checkpoints/summaries
- [MNIST](http://yann.lecun.com/exdb/mnist/) data set contains 60K training images and 10K test images at 28 x 28 px

#### Architecture
input -> conv -> max pool -> conv -> max pool -> dense (40% dropout) -> dense -> softmax

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

In [4]:
tf.logging.set_verbosity(tf.logging.INFO)

In [5]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN"""
    
    # input layer
    # features has shape [1, 28 * 28]
    # reshape it as [batch_size, height, width, channels]
    input_layer = tf.reshape(features['x'], [-1, 28, 28, 1])
    
    # conv layer 1
    # 32 5x5 filters, same padding, ReLU activation
    # input = [batch_size, 28, 28, 1]
    # output = [batch_size, 28, 28, 32]
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5,5],
        padding='same',
        activation=tf.nn.relu
    )
    
    # pool layer 1
    # max pool 2x2 filter w stride of 2 (reduces height/width 50%)
    # input = [batch_size, 28, 28, 32]
    # output = [batch_size, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2)
    
    # conv layer 
    # 64 5x5 filters, same padding, ReLU activation
    # input = [batch_size, 14, 14, 32]
    # output = [batch_size, 14, 14, 64]
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5,5],
        padding='same',
        activation=tf.nn.relu
    )
    
    # pool layer 2
    # max pool 2x2 filter w stride of 2
    # input = [batch_size, 14, 14, 64]
    # output = [batch_size, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)
    
    # fully connected layer
    # dense layer with 1024 neurons, ReLU activation
    # wrapped with dropout of 40% drop rate
    # first flatten the pool2 output into two dimensions [batch_size, 7 * 7 * 64]
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    # 3rd argument: Boolean which is true during training only
    # Required because dropout should only be applied during training phase
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # logits layer
    # 10 neuron fully connected layer
    # produces 10 logits, corresponding to 10 possible digits
    # output = [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    # converting logits into predictions with 2 formats
    # label given by logit with highest value
    # probability given by softmax activation
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step()
        )
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels,
            predictions=predictions["classes"]
        )
    }
    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=loss,
        eval_metric_ops=eval_metric_ops
    )
    

### Loading Data

In [6]:
def main(unused_argv):
    # Load training and eval data
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images # Returns np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images # Returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=20000,
        hooks=[logging_hook])

### Creating Estimator

In [7]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {}


In [ ]:
tf.app.run()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-20000
INFO:tensorflow:Saving checkpoints for 20001 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:loss = 0.166439, step = 20001
INFO:tensorflow:probabilities = [[ 0.00007454  0.99731761  0.00071517  0.00022771  0.00001481  0.00001781
   0.00003578  0.00088849  0.00066327  0.00004474]
 [ 0.0001478   0.00052849  0.47848022  0.49191704  0.00115809  0.00132548
   0.00014026  0.00007446  0.02578705  0.00044106]
 [ 0.00000044  0.00000539  0.00001885  0.00023323  0.04526043  0.00005018
   0.00000453  0.00041348  0.00031763  0.95369577]
 [ 0.00001988  0.00000026  0.000002    0.0001063   0.          0.9998641
   0.00000025  0.00000012  0.00000698  0.00000004]
 [ 0.00000624 

INFO:tensorflow:probabilities = [[ 0.00003446  0.00321405  0.00028038  0.99031222  0.0000273   0.00076888
   0.00000124  0.00020434  0.00190489  0.0032522 ]
 [ 0.00000007  0.          0.00000047  0.00000839  0.00000001  0.00002516
   0.00000002  0.00000181  0.99995196  0.00001215]
 [ 0.00000264  0.00000011  0.00000018  0.0000022   0.00016879  0.00000567
   0.          0.9854086   0.00000297  0.01440887]
 [ 0.000125    0.00004997  0.00651325  0.00409401  0.00043276  0.00004486
   0.00004354  0.00043736  0.09338728  0.89487189]
 [ 0.00002085  0.00000759  0.0000108   0.00195295  0.00000294  0.00766938
   0.0000201   0.00000007  0.99031198  0.00000333]
 [ 0.0097737   0.00081888  0.00044876  0.00070365  0.01395109  0.05022485
   0.92086494  0.0000042   0.00317566  0.00003428]
 [ 0.00008917  0.01020964  0.01002589  0.75394166  0.00005823  0.02552689
   0.00056721  0.00071746  0.19598331  0.00288054]
 [ 0.00000003  0.00000029  0.00034662  0.99424213  0.00000231  0.00006211
   0.00000004  0.00

INFO:tensorflow:global_step/sec: 5.86736
INFO:tensorflow:loss = 0.142066, step = 20101 (17.044 sec)
INFO:tensorflow:probabilities = [[ 0.00050983  0.93394822  0.00469796  0.00618964  0.00265766  0.00436929
   0.00076457  0.03369876  0.00735707  0.00580698]
 [ 0.99979109  0.00003094  0.00004239  0.00005273  0.00000086  0.00000833
   0.00002377  0.00000365  0.00002755  0.0000188 ]
 [ 0.0000282   0.00000014  0.00004514  0.00001011  0.0007724   0.00001038
   0.00000006  0.01984495  0.00005983  0.97922873]
 [ 0.00000166  0.00000004  0.00000227  0.00000017  0.9980427   0.0000185
   0.00010393  0.00001472  0.00003373  0.00178222]
 [ 0.00000384  0.00001001  0.00000143  0.00002049  0.01504975  0.00005378
   0.00000021  0.97826082  0.00000165  0.00659793]
 [ 0.0000218   0.00002649  0.00005289  0.00000509  0.00208106  0.00020351
   0.99750143  0.00000005  0.00010457  0.0000032 ]
 [ 0.01864995  0.0001309   0.96899039  0.00402932  0.00025117  0.00047204
   0.00073139  0.00000252  0.0066883   0.0000

INFO:tensorflow:probabilities = [[ 0.00000017  0.00000869  0.00000808  0.9993211   0.00001238  0.00025313
   0.00000013  0.00000694  0.0000741   0.00031537]
 [ 0.00104833  0.00006182  0.00074633  0.00008133  0.0006424   0.00050908
   0.99651515  0.0002059   0.00005343  0.0001362 ]
 [ 0.00001033  0.99147087  0.00016758  0.00017247  0.0000107   0.0000381
   0.00003366  0.0000777   0.00797314  0.00004543]
 [ 0.00000063  0.00000344  0.0002426   0.00341503  0.00000008  0.00000008
   0.          0.99225616  0.00000368  0.00407829]
 [ 0.99648499  0.00002041  0.00025914  0.00166589  0.00014409  0.00004073
   0.00101459  0.00007327  0.00005303  0.00024385]
 [ 0.99597651  0.00000264  0.00001395  0.00018174  0.00000002  0.00204169
   0.00001036  0.00174813  0.00000061  0.00002434]
 [ 0.00013937  0.00000387  0.00000592  0.000008    0.00024822  0.99868912
   0.00070096  0.00002117  0.00018268  0.00000071]
 [ 0.00045285  0.66941869  0.00089663  0.00771086  0.0002054   0.00215825
   0.00002949  0.026

INFO:tensorflow:global_step/sec: 6.1392
INFO:tensorflow:loss = 0.1494, step = 20201 (16.289 sec)
INFO:tensorflow:probabilities = [[ 0.00050325  0.00002121  0.0028068   0.48113272  0.00032833  0.00576124
   0.00005913  0.00001829  0.50558376  0.00378525]
 [ 0.00126221  0.0016811   0.02376901  0.4714596   0.001909    0.42285293
   0.00159572  0.00014018  0.0745023   0.00082794]
 [ 0.00000004  0.00000063  0.00001862  0.00005736  0.0014049   0.00002978
   0.00000003  0.00017278  0.00004006  0.99827576]
 [ 0.00672707  0.00133365  0.00162823  0.00040954  0.01057408  0.03089603
   0.91010916  0.00021287  0.03563891  0.00247039]
 [ 0.00000059  0.00001519  0.00001681  0.00000036  0.00003468  0.00055763
   0.99911267  0.          0.00026175  0.00000041]
 [ 0.99999213  0.          0.00000013  0.00000002  0.          0.00000128
   0.00000409  0.00000013  0.00000005  0.00000221]
 [ 0.00003136  0.00037899  0.00018014  0.00100958  0.437767    0.02715908
   0.00131841  0.00631045  0.00970269  0.516142

...

INFO:tensorflow:probabilities = [[ 0.00138548  0.00001099  0.00019249  0.000982    0.00352435  0.00464465
   0.2918534   0.00001412  0.6970287   0.00036377]
 [ 0.00008177  0.00011311  0.00046556  0.00071591  0.11368784  0.00028694
   0.00007778  0.01242835  0.0000996   0.87204319]
 [ 0.00005207  0.00000066  0.0000135   0.00000083  0.00000407  0.00005197
   0.99972731  0.          0.00014943  0.0000001 ]
 [ 0.99978715  0.          0.00000058  0.00000001  0.          0.00019364
   0.00001687  0.00000004  0.0000015   0.00000033]
 [ 0.00000024  0.00001345  0.99949372  0.00044772  0.00002239  0.00000949
   0.0000003   0.00000967  0.00000283  0.0000001 ]
 [ 0.          0.00002837  0.00020806  0.98827398  0.00000085  0.00000064
   0.          0.00542366  0.00487324  0.00119122]
 [ 0.0000828   0.00000631  0.00389516  0.00002905  0.96094775  0.00013184
   0.0007852   0.01434856  0.00159021  0.01818303]
 [ 0.00087049  0.01745261  0.00001814  0.1185357   0.00281545  0.60788542
   0.24857523  0.00

INFO:tensorflow:global_step/sec: 6.20572
INFO:tensorflow:loss = 0.0478738, step = 30301 (16.114 sec)
INFO:tensorflow:probabilities = [[ 0.00000002  0.00000001  0.000033    0.00030344  0.          0.          0.
   0.99964976  0.00000032  0.00001354]
 [ 0.00000312  0.00003673  0.99885416  0.00103719  0.00000072  0.00000002
   0.00000002  0.00000346  0.00006254  0.00000214]
 [ 0.00000181  0.          0.00000387  0.00007021  0.00337572  0.00000229
   0.00000019  0.00122307  0.00029939  0.99502349]
 [ 0.91617936  0.00114455  0.00998367  0.00096162  0.00231739  0.00614476
   0.03791568  0.00091072  0.01916187  0.00528034]
 [ 0.95969129  0.00000243  0.00050901  0.0022337   0.0001118   0.00006281
   0.00060103  0.00000573  0.03337108  0.00341111]
 [ 0.0000651   0.00000332  0.00257329  0.97101641  0.00000012  0.00160647
   0.          0.00003987  0.02351811  0.00117734]
 [ 0.00012133  0.00336864  0.00013073  0.00003701  0.00003655  0.24904938
   0.57112098  0.00000007  0.17613219  0.00000311]


INFO:tensorflow:probabilities = [[ 0.00000004  0.00001037  0.00006119  0.99968445  0.00000004  0.00022854
   0.          0.00000032  0.00000014  0.00001487]
 [ 0.00000131  0.00001115  0.00001296  0.00030692  0.00280649  0.00001288
   0.00000037  0.00119009  0.00059583  0.99506193]
 [ 0.00000002  0.00000002  0.00000005  0.00000005  0.99954396  0.00000064
   0.00000016  0.0000687   0.00000375  0.00038265]
 [ 0.99920565  0.          0.00000201  0.0000001   0.          0.00078991
   0.00000028  0.00000196  0.00000003  0.00000011]
 [ 0.00000128  0.00000014  0.00000894  0.00003073  0.00001139  0.00055945
   0.00000848  0.00000908  0.99857652  0.000794  ]
 [ 0.00003166  0.0005168   0.00143201  0.0035119   0.0000011   0.0000043
   0.00000206  0.00000015  0.99443758  0.00006249]
 [ 0.00000324  0.00000481  0.0000566   0.00009234  0.00011279  0.00012184
   0.00002395  0.00000201  0.99935645  0.00022605]
 [ 0.00000634  0.00000007  0.00032627  0.0000634   0.00000086  0.0000404
   0.00000204  0.0000

INFO:tensorflow:global_step/sec: 6.19458
INFO:tensorflow:loss = 0.158883, step = 30401 (16.143 sec)
INFO:tensorflow:probabilities = [[ 0.00000519  0.00000715  0.00004927  0.00013346  0.01144801  0.00006822
   0.00000031  0.00227882  0.00004208  0.98596746]
 [ 0.00000023  0.00000135  0.00000013  0.00000921  0.41319707  0.00002641
   0.00000044  0.00066469  0.00491044  0.58119005]
 [ 0.00000311  0.00000028  0.00000069  0.00069747  0.0039025   0.00003671
   0.00000002  0.0000943   0.0001808   0.99508411]
 [ 0.0000188   0.00000678  0.00316283  0.99614775  0.00000008  0.00007639
   0.0000001   0.00000009  0.00057505  0.00001216]
 [ 0.00001144  0.99845195  0.0000605   0.00000174  0.00006977  0.00003129
   0.00054125  0.00003131  0.00075045  0.00005024]
 [ 0.00000192  0.99934345  0.00022365  0.00004965  0.00002495  0.00000125
   0.0000287   0.00005369  0.0002639   0.00000892]
 [ 0.00000064  0.00000004  0.00000029  0.00000813  0.          0.99924302
   0.00000034  0.          0.00074673  0.000

INFO:tensorflow:probabilities = [[ 0.99956149  0.00000041  0.00004838  0.00000093  0.00000013  0.00024831
   0.00000684  0.000088    0.00000098  0.0000445 ]
 [ 0.00008575  0.00015282  0.00036173  0.0000393   0.00010751  0.00196666
   0.99591762  0.00000017  0.00136562  0.00000284]
 [ 0.00000186  0.99927896  0.00001008  0.00002848  0.0003608   0.00000187
   0.00005189  0.00007571  0.00017955  0.00001075]
 [ 0.00058286  0.00010734  0.00005944  0.00000487  0.00060769  0.06518565
   0.93322992  0.00000079  0.00021346  0.00000798]
 [ 0.00013368  0.00039133  0.00028219  0.00084631  0.03699356  0.00192425
   0.00001058  0.08918177  0.00158625  0.86865002]
 [ 0.00048577  0.00126624  0.00630161  0.85841447  0.00087448  0.00318803
   0.0000057   0.04495729  0.00824105  0.07626539]
 [ 0.98697376  0.00016179  0.00896981  0.00027373  0.00013296  0.00036983
   0.00061224  0.00017313  0.00036585  0.00196688]
 [ 0.00000697  0.00035236  0.98953187  0.01004484  0.00000005  0.00006106
   0.00000026  0.00

In [ ]:
if __name__ == "__main__":
  tf.app.run()